## Overview
This notebook processes Toronto road network and collision data to create a timeseries dataset for an accident prediction model for the Toronto downtown core. It also contains a traffic accident risk prediction model that is trained on this dataset for the Toronto downtown core.

The dataset sources are:
1. [NRN Ontario GeoPackage](https://open.canada.ca/data/en/dataset/3d282116-e556-400c-9306-ca1a3cada77f/resource/d07a84dd-863c-4d60-9c08-0b33b6120427)
2. [Toronto neighbourhood spatial data](https://open.toronto.ca/dataset/neighbourhoods/)
3. [Toronto collision dataset](https://open.toronto.ca/dataset/police-annual-statistical-report-traffic-collisions/)

## Part 0: Notebook setup

First installing useful modules:

In [ ]:
!pip install geopandas shapely torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.1 MB/s eta 0:00:00


Next, importing necessary modules:

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
import torch_geometric as pyg
from torch_geometric.nn import GCNConv
import matplotlib.pyplot as plt

from datetime import datetime
from zoneinfo import ZoneInfo

import shapely
from shapely.geometry import Point

## Part 1: Preprocessing GeoDataFrames

### 1.1: Extracting raw data

Mounting google drive and changing working directory:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Note: paste your own working directory here and then run the cell proceed with the next steps

%cd "/content/drive/MyDrive/2. UofT/2024-2025 (PEY)/Summer 2025/APS360 (2025)/APS360 2025 Project"

[Errno 2] No such file or directory: '/content/drive/MyDrive/2. UofT/2024-2025 (PEY)/Summer 2025/APS360 (2025)/APS360 2025 Project'
/content


Loading raw datasets and converting to GeoDataFrames

In [ ]:
# Available layers: 'NRN_ON_18_0_TOLLPOINT' (default), 'NRN_ON_18_0_FERRYSEG', 'NRN_ON_18_0_JUNCTION',
# 'NRN_ON_18_0_ROADSEG', 'NRN_ON_18_0_BLKPASSAGE', 'NRN_ON_18_0_STRPLANAME', 'NRN_ON_18_0_ADDRANGE'
roads_gpd = gpd.read_file("./3. Data and processing/DTC Road Network Data/NRN_RRN_ON_GPKG/NRN_ON_18_0_GPKG_en.gpkg", \
                          layer = 'NRN_ON_18_0_ROADSEG')

# Making neighbourhoods gpd:
nbhds_gpd = gpd.read_file("./3. Data and processing/DTC Road Network Data/Neighbourhoods - 4326.gpkg")

# Making collisions gpd:
collisions_gpd = gpd.read_file('./3. Data and processing/Toronto Collision Data/Traffic Collisions - 4326.gpkg')

DataSourceError: ./3. Data and processing/DTC Road Network Data/NRN_RRN_ON_GPKG/NRN_ON_18_0_GPKG_en.gpkg: No such file or directory

### 1.2: Clipping datasets to Toronto downtown core

In [ ]:
##################################################################
## creating downtown core polygon
##################################################################

# Making list of neighbourhoods in Toronto downtown core
dtc_nbhds_list = ['Annex', 'University', 'Kensington-Chinatown', 'Wellington Place', 'Bay-Cloverhill', \
             'Yonge-Bay Corridor', 'Church-Wellesley', 'Downtown Yonge East', 'North St.James Town', \
             'Cabbagetown-South St.James Town', 'Moss Park', 'Regent Park', 'Harbourfront-CityPlace']

# Filtering to downtown core neighhbourhoods
dtc_nbhds_gpd = nbhds_gpd[nbhds_gpd['AREA_NAME'].isin(dtc_nbhds_list)]

# Dissolving interior boundaries to create a single polygon
dtc_nbhds_gpd = dtc_nbhds_gpd.dissolve()


# Reprojecting all gpds to match NRN road network (just in case)
dtc_nbhds_gpd = dtc_nbhds_gpd.to_crs(epsg=4617)
colls_gpd = collisions_gpd.to_crs(epsg=4617)


##################################################################
## using polygon to clip the NRN road network
##################################################################

roads_gpd_clipped = gpd.clip(roads_gpd, dtc_nbhds_gpd['geometry'], keep_geom_type=True)
colls_gpd_clipped = gpd.clip(colls_gpd, dtc_nbhds_gpd['geometry'], keep_geom_type=True)

### 1.3: Processing time data into [year, month, day, hour] format

The goal of this section is to have a consistent way of denoting the timepoint associated with each row in each GPD.

In [ ]:
##################################################################
## removing extraneous columns from GPDs
##################################################################

colls_gpd_clipped.drop(columns = ["_id", "OCC_MONTH", "OCC_DOW", "OCC_YEAR", "DIVISION", "HOOD_158", "NEIGHBOURHOOD_158", "LONG_WGS84", "LAT_WGS84", "FATALITIES"], \
                       inplace = True)

In [ ]:
##################################################################
## replacing time-related columns with columns of format [YEAR, MONTH, DAY, HOUR] for collisions
##################################################################

## For colls_gpd_clipped, we can use the datetime and zoneinfo modules to get the necessary date and hour info

# Making custom get_coll_time_info() function that creates a pd Series of time information
def get_coll_time_info(occ_date):

  date_EST = datetime.fromtimestamp(int(occ_date) / 1000, ZoneInfo("America/Toronto"))

  return pd.Series([date_EST.year, date_EST.month, date_EST.day])


# Making coll_OCC_convert() funciton to convert OCC_DATE data into
def coll_OCC_convert(coll_df):

  temp_df = pd.DataFrame()
  temp_df[['YEAR', 'MONTH', 'DAY']] = coll_df["OCC_DATE"].apply(get_coll_time_info)

  coll_df.insert(coll_df.columns.get_loc('OCC_DATE'), 'YEAR', temp_df['YEAR'])
  coll_df.insert(coll_df.columns.get_loc('OCC_DATE'), 'MONTH', temp_df['MONTH'])
  coll_df.insert(coll_df.columns.get_loc('OCC_DATE'), 'DAY', temp_df['DAY'])
  coll_df.pop("OCC_DATE")


# Applying functions to colls_gpd_clipped to modify it in place
coll_OCC_convert(colls_gpd_clipped)

# Renaming OCC_HOUR col to HOUR, in place, and turning str instries into int
colls_gpd_clipped.rename(columns = {'OCC_HOUR': 'HOUR'}, inplace = True)      # Renames rows in place
colls_gpd_clipped['HOUR'] = colls_gpd_clipped['HOUR'].apply(lambda x: int(x))


# Removing duplicate entries and reindexing
colls_gpd_clipped.drop_duplicates(subset = ['YEAR', 'MONTH', 'DAY', 'HOUR', 'geometry'], keep = 'first', inplace = True)
colls_gpd_clipped = colls_gpd_clipped.reset_index(drop = True)

# Converting 'NO' and 'YES' entries into 0s and 1s
colls_gpd_clipped.replace({'NO': 0, 'YES': 1, 'N/R': 0}, inplace = True)   # Modifies DF in place

## Part 2: Creating Graphs, Timeseries, and Labels

### 2.1: Extracting nodes and edges from downtown core road network

In [ ]:
##################################################################
## Extracting endpoints (nodes) from the downtown core GPD
##################################################################

# Retrieving endpoints for each geometry
endpts = roads_gpd_clipped['geometry'].boundary

# Turning multipoints into single points, removing duplicates, and resetting index
endpts = endpts.explode().drop_duplicates()   # endpts now represents our nodes

# Convert the endpts GeoSeries to a GeoDataFrame
endpts_gpd = gpd.GeoDataFrame(geometry = endpts)
endpts_gpd = endpts_gpd.reset_index(drop = True)   # reorders endpts so that index starts from 0

# Creating endpts dictionary to map a given coordinate (Point()) to a node number/order
node_to_idx = {(pt[0]): ind for ind, pt in endpts_gpd.iterrows()}

In [ ]:
##################################################################
## Extracting edges and edge attributes from roads_gpd_clipped and node_to_idx
##################################################################

# Creating edge list using node_to_idx mapping
edges = []
edge_atts = []
edge_weights = []

for i, row in roads_gpd_clipped.explode().iterrows():

  # Creating undirected edge list
  start = Point(row['geometry'].coords[0])   # Wrapping in Point() since output is a tuple
  end = Point(row['geometry'].coords[-1])    # Wrapping in Point() since output is a tuple

  u_pos = node_to_idx[start]
  v_pos = node_to_idx[end]

  edges.append((u_pos, v_pos))
  edges.append((v_pos, u_pos))

  # Creating edge attribute matrix
  edge_atts.append([row['PAVSTATUS'], row['ROADCLASS'], row['NBRLANES'], row['TRAFFICDIR'], row['geometry'].length])

  # Creatng edge weight vector for compatibility with PyG GCN architecture
  edge_weights.append(row['geometry'].length)


# Duplicating the length values to match the number of directed edges
edge_weights = edge_weights + edge_weights

### 2.2: Snapping Collision Geometries to Nearest Nodes (Endpoints)

In [ ]:
##################################################################
## Creating feature_to_index mapping
##################################################################

# Note: this mapping is arbitrary
coll_features = ['INJURY_COLLISIONS', 'FTR_COLLISIONS', 'PD_COLLISIONS', 'AUTOMOBILE',	'MOTORCYCLE',	'PASSENGER', 'BICYCLE', 'PEDESTRIAN']

features = coll_features.copy()

features_to_idx = {feats: i for i, feats in enumerate(features)}

In [ ]:
##################################################################
## Preparing DFs by snapping geometries to node geometries
##################################################################

# Using shapely STRtree to build R-tree of endpoint geometries
tree = shapely.STRtree(endpts)

# Getting target indices of nearest points to speed_avg
target_coll_idxs = tree.nearest(colls_gpd_clipped['geometry'])

# Finding snapped speed geometries from target indices
coll_snapped = tree.geometries.take(target_coll_idxs)

# Converting to GeoDataFrame
coll_snapped = gpd.GeoDataFrame(geometry = coll_snapped, crs = 'EPSG:4617')


##################################################################
## Replacing geometries in coll GPD with snapped geometries
##################################################################

# Updating coll GPD
colls_gpd_clipped.pop("geometry")
colls_gpd_clipped['geometry'] = coll_snapped['geometry']

### 2.3: Creating Timeseries and Ground Truth Labels

In [ ]:
##################################################################
## Creating list of GPDs where each entry is all collision events in a given hour
##################################################################

coll_grouped = colls_gpd_clipped.groupby(['YEAR', 'MONTH', 'DAY', 'HOUR'])   # Groups dataframe into hour-level spatial chunks

ordered_coll_list = []

for i in list(coll_grouped.groups):                      # Gets key from list of keys
  ordered_coll_list.append(coll_grouped.get_group(i))    # Uses that key to access value (which are the gpd indices) and uses that as arg for .get_group()

coll_timeseries = pd.concat(ordered_coll_list).reset_index(drop = True)


##################################################################
## Creating timepoint_to_index mapping
##################################################################

coll_timeseries_grouped = coll_timeseries.groupby(['YEAR', 'MONTH', 'DAY', 'HOUR'])
timepoints_to_idx = {pt: idx for idx, pt in enumerate(coll_timeseries_grouped.groups.keys())}

In [ ]:
##################################################################
## Creating timeseries and labels array
##################################################################

timeseries = np.zeros(shape = (len(timepoints_to_idx), len(endpts), len(features)), dtype = np.float64)
timeseries_labels = np.zeros(shape = (len(timepoints_to_idx), len(endpts), 1), dtype = np.float64)


##################################################################
## Populating timeseries with processed collision data and labels
##################################################################

# Iterating through coll_timeseries and updating timeseries and labels entries
count = 0
for i in coll_timeseries[['YEAR', 'MONTH', 'DAY', 'HOUR']].iterrows():

  # Checking if timepoint tuple is in timepoints_to_idx mapping:
  if tuple(i[1]) in timepoints_to_idx.keys():

    # If the above is True, we update the features at the correct time and node index
    timeseries[timepoints_to_idx[tuple(i[1])]][node_to_idx[coll_timeseries['geometry'].iloc[count]]][:] \
    = np.array(coll_timeseries.iloc[count][features], dtype = np.float64)

    # If the above is True, we update the labels at the correct time and node index
    timeseries_labels[timepoints_to_idx[tuple(i[1])]][node_to_idx[coll_timeseries['geometry'].iloc[count]]] = 1

  count += 1

## Part 3: Preparing Input Data

### 3.1: Creating torch.tensors of Features and Labels

In [ ]:
##################################################################
## Creating [2, E] edge tensor
##################################################################

# Creating matrix of shape [2, E] for edges
sources, targets = zip(*edges)
edge_index_tensor = torch.tensor([sources, targets], dtype=torch.long)

# Turning edge weights into torch tensor
edge_weight_tensor = torch.tensor(edge_weights, dtype = torch.float)


##################################################################
## Creating features (input) and labels tensors
##################################################################

# turning node feature timeseries into torch tensor
inputs = torch.tensor(timeseries, dtype = torch.float)

# turning labels into torch tensor
labels = torch.tensor(timeseries_labels, dtype = torch.float)

In [ ]:
##################################################################
## Creating sliding window of input tensors and associated ground truth labels
##################################################################

# Choosing window size of 7:
window_size = 7

# Choosing a window size of 7 to represent 'weeks' and permuting to have them in correct position
input_w = inputs.unfold(dimension = 0, size = window_size, step = 1)
input_w = torch.permute(input_w, (0, 3, 1, 2))

# Removing last window (since there is no available ground truth)
input_w = input_w[:-1]

# Truncating labels to all occurrences at time t+1 relative to the end of each window
labels_w = labels[window_size:]

In [ ]:
##################################################################
## Reshaping the input and label windows to concatenate features across time
##################################################################

# Extracting dimensions
w, t, n, f = input_w.shape

# Permuting input_w
input_w = torch.permute(input_w, (0, 2, 1, 3))
input_w.shape

# Reshaping to [w, n, t*f]
input_w = torch.reshape(input_w, (w, n, t*f))

### 3.2: Creating Training, Validation, and Test Sets

In [ ]:
##################################################################
## loading all data into a PyG Data object
##################################################################

data_list = []

for i in range(len(input_w)):
  data_list.append(pyg.data.Data(x = input_w[i], edge_index = edge_index_tensor, edge_attr = edge_weight_tensor, y = labels[i]))

In [ ]:
##################################################################
## loading all data into a PyG Data object
##################################################################

# Setting seed for reproducible shuffling
torch.manual_seed(1000)

split1 = int(0.7 * len(data_list))
split2 = int(0.85 * len(data_list))

train_data = data_list[:split1]
val_data = data_list[split1:split2]
test_data = data_list[split2:]

batch_size = 20

train_loader = pyg.loader.DataLoader(train_data, batch_size = batch_size, shuffle = True)
val_loader = pyg.loader.DataLoader(val_data, batch_size = batch_size, shuffle = False)
test_loader = pyg.loader.DataLoader(test_data, batch_size = batch_size, shuffle = False)

## Part 4: Building and Training ANN-GCN Model

### 4.1: Building ANN-GCN Model

In [ ]:
##################################################################
## Creating preliminary ANN-GCN hybrid
##################################################################

class ANN_GCN(torch.nn.Module):
  def __init__(self, in_channels, hidden_linear, out_linear, hidden_channels, out_channels):
    super(ANN_GCN, self).__init__()
    self.name = 'ANN-GCN'

    torch.manual_seed(10000)

    self.linear1 = nn.Linear(in_channels, hidden_linear)
    self.linear2 = nn.Linear(hidden_linear, out_linear)
    self.conv1 = GCNConv(out_linear, hidden_channels)
    self.conv2 = GCNConv(hidden_channels, out_channels)
    self.linear3 = nn.Linear(out_channels, 1)


  def forward(self, x, edge_index, edge_weight):
    x = F.relu(self.linear1(x))
    x = F.relu(self.linear2(x))
    x = F.relu(self.conv1(x, edge_index, edge_weight))
    x = F.relu(self.conv2(x, edge_index, edge_weight))
    x = self.linear3(x)

    return x

### 4.2: Defining Training and Plotting Functions

In [ ]:
##################################################################
## Creating training function
##################################################################

def train_net(net, train_loader, val_loader, learning_rate, epochs):

    # Moving net to GPU if available
    if torch.cuda.is_available():
      net = net.cuda()
      pos_weight_tensor = torch.tensor(2000).cuda()  # Move pos_weight to GPU

    # Using BCEWithLogitsLoss for binary classification
    criterion = nn.BCEWithLogitsLoss(pos_weight = pos_weight_tensor if torch.cuda.is_available() else torch.tensor(2000)) # Use the calculated pos_weight
    optimizer = optim.Adam(net.parameters(), lr = learning_rate)

    # training the network and recording train and val accuracies
    train_accuracy = np.zeros(epochs)
    val_accuracy = np.zeros(epochs)

    for epoch in range(epochs):

      num_correct = 0
      total = 0

      net.train() # Set the model to training mode

      for i, batch in enumerate(train_loader):

        node_feats = batch.x
        edge_atts = batch.edge_attr
        edge_index = batch.edge_index
        labels = batch.y

        # Reshaping labels to match output shape [batch_size * num_nodes, 1]
        labels = labels.view(-1, 1)


        # Moving tensors to the GPU if available
        if torch.cuda.is_available():
          node_feats = node_feats.cuda()
          edge_atts = edge_atts.cuda()
          edge_index = edge_index.cuda()
          labels = labels.cuda()

        # doing forward and backward pass
        optimizer.zero_grad()
        outputs = net(node_feats, edge_index, edge_atts)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # computing the training accuracy
        prediction = (torch.sigmoid(outputs) > 0.5).float()
        comparison = torch.eq(prediction, labels)
        num_correct += int(torch.sum(comparison))
        total += labels.size(0)   # Using the actual number of nodes in the batch


      train_accuracy[epoch] = num_correct / total
      val_accuracy[epoch] = accuracy(net, val_loader)

      # Saving model checkpoint
      model_path = get_model_name(net.name, batch_size, learning_rate, epoch)
      torch.save(net.state_dict(), model_path)

      # Printing results:
      print("Epoch: {}, training accuracy: {}, validation accuracy: {}".format(epoch + 1, train_accuracy[epoch], \
            val_accuracy[epoch]))

    epochs = np.arange(1, epochs + 1)

    return train_accuracy, val_accuracy, epochs


##################################################################
## Creating accuracy function
##################################################################

def accuracy(net, loader):
    """
    This function evaluates a given model iteration on a dataset and returns the accuracy
    for that dataset.
    (nn.Module), (DataLoader) --> (int)
    """
    net.eval() # Set the model to evaluation mode
    num_correct = 0
    total = 0

    # evaluating the model on the given dataset and computing the accuracies
    with torch.no_grad(): # Disable gradient calculation for evaluation
        for i, batch in enumerate(loader):

          node_feats = batch.x
          edge_atts = batch.edge_attr
          edge_index = batch.edge_index
          labels = batch.y   # Removing unsqueeze(1)

          # Reshaping labels to match output shape [batch_size * num_nodes, 1]
          labels = labels.view(-1, 1)

          # Moving tensors to the GPU if available
          if torch.cuda.is_available():
            node_feats = node_feats.cuda()
            edge_atts = edge_atts.cuda()
            edge_index = edge_index.cuda()
            labels = labels.cuda()

          net_output = net(node_feats, edge_index, edge_atts)

          # Computing the training accuracy
          prediction = (torch.sigmoid(net_output) > 0.5).float()
          comparison = torch.eq(prediction, labels)
          num_correct += int(torch.sum(comparison))
          total += labels.size(0)    # Using the actual number of nodes in the batch

    return num_correct / total


##################################################################
## Creating get_model_name function
##################################################################

def get_model_name(name, batch_size, learning_rate, epoch):

    path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(name, batch_size, learning_rate, epoch)
    return path


##################################################################
## Creating plotting function
##################################################################

def plot_training_curve(training_accuracy, val_accuracy, epochs):
    """ Plots the training curve for a model run, given training and
    validation accuracy.
    """
    import matplotlib.pyplot as plt

    plt.plot(epochs, training_accuracy, label = "Training accuracy")
    plt.plot(epochs, val_accuracy, label = "Validation accuracy")
    plt.xlabel("Epoch number")
    plt.ylabel("Accuracy")
    plt.legend(loc='best')
    plt.show()

### 4.3: Training ANN-GCN Model

In [ ]:
##################################################################
## Trianing model and plotting training curve
##################################################################

# Defining model
model = ANN_GCN(56, 16, 8, 12, 1)

# Training and plotting
train_acc, val_acc, epochs = train_net(model, train_loader, val_loader, 0.0001, 10)
plot_training_curve(train_acc, val_acc, epochs)

In [ ]:
##################################################################
## Counting number of positive and negative samples in dataset
##################################################################

flat_labels = labels.view(-1)

all_neg_acc = 1 - (torch.sum(flat_labels).item() / len(flat_labels))

print("Approx. accuracy if all predictions were negative:", all_neg_acc)

### 4.4: Evaluating Best Model on Test Set

In [ ]:
##################################################################
## Loading best model
##################################################################

best_checkpoint_path = get_model_name(model.name, 20, 0.0001, 9)

best_model = ANN_GCN(56, 16, 8, 12, 1)      # Making a new model to load best parameters into
best_model.load_state_dict(torch.load(best_checkpoint_path))

In [ ]:
##############################
# Evaluating model on the test set
##############################

# Moving best_model to GPU if available
if torch.cuda.is_available():
  best_model = best_model.cuda()

test_acc = accuracy(best_model, test_loader)
print("Test accuracy: {:.2f}%".format(test_acc * 100))